In [ ]:
import csv

# Define the path to the text file
log_txt_file = "0909-ori-20v0.txt"

try:
    # Read the log data from the text file
    with open(log_txt_file, "r") as file:
        lines = file.readlines()

    # Parse the lines and prepare the CSV data
    csv_data = [["Time", "EventType", "x", "y", "z", "r", "n"]]  # CSV headers

    for line in lines:
        line = line.strip()
        if line and line[0].isdigit():  # Only process lines with events
            parts = line.split("\t")
            # if parts[1] == "p":
            csv_data.append(parts + [""] * (6 - len(parts)))  # Ensure 6 columns in the CSV
            # else:
            #     csv_data.append(parts + [""] * (3 - len(parts)))

    # Write the parsed data to a CSV file
    csv_filename = "0909-ori-20v0.csv"
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)

    print(f"CSV file saved as {csv_filename}")

except FileNotFoundError:
    print(f"Error: The file {log_txt_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


In [1]:
import pandas as pd

# Load the CSV file into a dataframe
df = pd.read_csv('0925-fixed-20v0.csv', sep=',', header=None, engine='python')

# Naming the columns
df.columns = ['Time', 'EventType', 'Value1', 'Value2', 'Value3', 'Value4', 'Value5']

# Preprocess the CSV:
# For events 'n', 'h', and 'f', keep only the event type and trial number (Value1).
# For event 'p', keep all the position and rotation data (x, y, z, and rotation).
def preprocess_event(row):
    event_type = row['EventType']
    if event_type in ['n', 'h', 'f']:  # Only keep 'Value1' for these events
        return pd.Series([row['Time'], event_type, row['Value1'], None, None, None, None])
    elif event_type == 'p':  # Keep all position and rotation data
        return pd.Series([row['Time'], event_type, None, row['Value1'], row['Value2'], row['Value3'], row['Value4']])
    else:
        return row  # Unhandled events (if any)

print("Shape of df_processed after applying preprocess_event:", df_processed.shape)
print("First few rows of df_processed:", df_processed.head())

# Apply the preprocessing function to each row
df_processed = df.apply(preprocess_event, axis=1)

# Rename columns for clarity
df_processed.columns = ['Time', 'EventType', 'TrialNumber', 'x', 'y', 'z', 'rotation']

# Convert the 'Time' column to datetime for further calculations
df_processed['Time'] = pd.to_datetime(df_processed['Time'], format='%H:%M:%S.%f')

# Save the preprocessed CSV for future use (optional)
df_processed.to_csv('preprocessed_data.csv', index=False)

print("Preprocessing complete, saved as 'preprocessed_data.csv'")


ValueError: Length mismatch: Expected axis has 14 elements, new values have 7 elements